In [3]:
# League of Legends Game State Analyzer with YOLO Detection
import cv2
import pytesseract
import os
import time
import numpy as np
from PIL import Image
import pandas as pd
from ultralytics import YOLO

# Configuration
OUTPUT_DIR = "yolo_extracted_frames"
MAX_PROCESSING_SECONDS = 900  # 15 minutes
VIDEO_PATH = "videos/1.mp4"
ANALYSIS_INTERVAL = 1  # Seconds between frames
OUTPUT_IMAGES = True  # Set False to disable image saving
MODEL_PATH = "yolov8n.pt"  # Trained YOLOv8 model path

# Region of Interest Configurations (1920x1080 resolution)
UI_CONFIG = {
    "gold": {
        "roi": (1170, 1045, 70, 30),
        "threshold": 180,
        "digits": 4
    },
    "minions": {
        "roi": (1775, 0, 40, 30),
        "threshold": 190,
        "digits": 3
    },
    "skills": {
        "Q": (730, 950, 60, 60),
        "W": (795, 950, 60, 60),
        "E": (860, 950, 60, 60),
        "R": (925, 950, 60, 60)
    }
}

# Cooldown detection parameters
BLUE_HSV_LOWER = np.array([90, 50, 50], dtype=np.uint8)
BLUE_HSV_UPPER = np.array([120, 255, 255], dtype=np.uint8)
CD_THRESHOLD = 0.2

# Tesseract Setup (Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

class CooldownDetector:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.digit_map = {
            0: '0', 1: '1', 2: '2', 3: '3', 4: '4',
            5: '5', 6: '6', 7: '7', 8: '8', 9: '9'
        }
    
    def detect_digits(self, skill_img):
        results = self.model(skill_img)
        digits = []
        
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = box.conf.item()
            cls = int(box.cls.item())
            
            if conf > 0.5:  # Confidence threshold
                digit_roi = skill_img[y1:y2, x1:x2]
                digit = self.ocr_digit(digit_roi)
                digits.append(digit)
        
        return ''.join(digits) if digits else None
    
    def ocr_digit(self, digit_roi):
        processed = cv2.cvtColor(digit_roi, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(processed, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        text = pytesseract.image_to_string(thresh, config='--psm 10 --oem 3')
        return text.strip() if text else '?'

def preprocess_image(roi):
    """Enhanced image optimization pipeline"""
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    _, thresh = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def extract_digits(processed_img, config):
    """OCR execution with validation"""
    custom_config = f'--psm 6 --oem 3 -c tessedit_char_whitelist=0123456789'
    try:
        text = pytesseract.image_to_string(Image.fromarray(processed_img), config=custom_config)
        filtered = ''.join(filter(str.isdigit, text)).lstrip('0')
        return filtered.zfill(config["digits"]) if filtered else '0'*config["digits"]
    except Exception as e:
        print(f"OCR Error: {str(e)}")
        return 'NULL'

def detect_cooldown(skill_roi, detector):
    """Detect cooldown state using YOLO model"""
    hsv = cv2.cvtColor(skill_roi, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, BLUE_HSV_LOWER, BLUE_HSV_UPPER)
    coverage = np.sum(mask > 0) / (skill_roi.size / 3)
    
    if coverage > CD_THRESHOLD:
        detected_digits = detector.detect_digits(skill_roi)
        return detected_digits or "CD", skill_roi
    
    return "Ready", None

def process_frame(frame, frame_count, detector):
    """Complete frame processing workflow"""
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    frame_dir = os.path.join(OUTPUT_DIR, f"frame_{frame_count}")
    marked_frame = frame.copy()
    results = {}
    
    if OUTPUT_IMAGES:
        os.makedirs(frame_dir, exist_ok=True)
        cv2.imwrite(os.path.join(frame_dir, "original.jpg"), frame)

    # Process gold and minions
    for element in ["gold", "minions"]:
        cfg = UI_CONFIG[element]
        x, y, w, h = cfg["roi"]
        roi = frame[y:y+h, x:x+w]
        
        processed = preprocess_image(roi)
        results[element] = extract_digits(processed, cfg)
        
        if OUTPUT_IMAGES:
            cv2.imwrite(os.path.join(frame_dir, f"{element}_processed.jpg"), processed)
            cv2.rectangle(marked_frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Process skills using YOLO model
    skill_results = {}  # Ensure this is initialized as a dictionary
    for skill, roi in UI_CONFIG["skills"].items():
        x, y, w, h = roi
        skill_roi = frame[y:y+h, x:x+w]
        
        cd_state, processed_img = detect_cooldown(skill_roi, detector)
        skill_results[skill] = cd_state  # Store as key-value pair
        
        # Visualization with YOLO detections
        color = (255, 0, 0) if cd_state not in ["Ready"] else (0, 255, 0)
        cv2.rectangle(marked_frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(marked_frame, cd_state, (x+5, y+20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # # Draw digit bounding boxes
        # if cd_state not in ["Ready"]:
        #     results = detector.model(skill_roi)
        #     for box in results[0].boxes:
        #         x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        #         cv2.rectangle(marked_frame, 
        #                     (x + x1, y + y1), 
        #                     (x + x2, y + y2), 
        #                     (0, 255, 0), 1)

    if OUTPUT_IMAGES:
        cv2.imwrite(os.path.join(frame_dir, "marked.jpg"), marked_frame)

    return {
        "timestamp": timestamp,
        **results,
        **skill_results,
        "frame_path": frame_dir
    }

def video_processing(detector):
    """Main processing loop with time constraint"""
    start_time = time.time()
    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * ANALYSIS_INTERVAL)
    
    dataset = []
    frame_count = 0
    
    while cap.isOpened() and (time.time() - start_time) <= MAX_PROCESSING_SECONDS:
        ret, frame = cap.read()
        if not ret: break
        
        if frame_count % frame_interval == 0:
            frame_data = process_frame(frame, frame_count, detector)
            log_entry = (f"Frame {frame_count} | "
                        f"Gold: {frame_data['gold']} | "
                        f"Minions: {frame_data['minions']} | "
                        f"Q: {frame_data['Q']} | "
                        f"W: {frame_data['W']} | "
                        f"E: {frame_data['E']} | "
                        f"R: {frame_data['R']}")
            print(log_entry)
            dataset.append(frame_data)
        
        frame_count += 1
    
    pd.DataFrame(dataset).to_csv(os.path.join(OUTPUT_DIR, "analysis_report.csv"), index=False)
    cap.release()
    print(f"Processing completed. Results saved to {OUTPUT_DIR}")

if __name__ == "__main__":
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    detector = CooldownDetector(MODEL_PATH)
    video_processing(detector)

Frame 0 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 28 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 56 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 84 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 112 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 140 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 168 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 196 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 224 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 252 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 280 | Gold: 0500 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 308 | Gold: 0000 | Minions: 000 | Q: Ready | W: Ready | E: Ready | R: Ready
Frame 336 | Gold: 000